In [ ]:
from climateforcing.utils import mkdir_p
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pickle
import warnings

In [ ]:
9.4e8/5.1e18 * 1e9

In [ ]:
IIRF_HORIZON=100
IIRF_MAX = 99.95
GAS_BOX_AXIS=4
N_GAS_BOXES=4
TIME_AXIS=0
M_ATMOS = 5.1352e18 # mass of atmosphere, kg
RADIUS_EARTH = 6371000 # radius of atmosphere, m

In [ ]:
# for aerosols, convert mg m-2 to Tg by a factor of about a half

In [ ]:
# burden of sulfate in kilograms (is this S, SO2 or SO4?)
# Myhre aerocom: 1.78 mg m-2
4 * np.pi * RADIUS_EARTH**2 * 1.78 / 100000
#1   1           m2          g m-2  kg mg-1   = kg

# then expressed in TgSO4
4 * np.pi * RADIUS_EARTH**2 * 1.78 / 1e15

# Pham et al. 2005: global sulfur burden (in S) was 0.47 TgS in 1990 (470 Gg = 470 Mt)
# SO4 to S conversion is approx divide 1.5 so this 0.9 TgSO4 is 0.6 TgS, not far from Pham

In [ ]:
#AR4:                0.58 mg NO3 / m-2, 
#Myhre 2013 aerocom: 0.56 mg NO3 / m-2, 
4 * np.pi * RADIUS_EARTH**2 * 0.56 / 1e15   # global TgNO3 burden

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']
n_scenarios = len(scenarios)

In [ ]:
timebounds = np.arange(1750, 2501.5, 1)
timesteps  = np.arange(1750.5, 2501, 1)
timestep   = 1

In [ ]:
n_timesteps = len(timesteps)
n_timebounds = len(timebounds)

In [ ]:
wmghgs = [
    "C2F6",
    "C3F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "cC4F8",  # not standard PubChem but used extensively in AR6
    "CCl4",
    "CF4",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "CFC-11",
    "CFC-12",
    "CH2Cl2",
    "CH3Br",
    "CH3CCl3",
    "CH3Cl",
    "CH4",
    "CHCl3",
    "CO2",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "HCFC-141b",
    "HCFC-142b",
    "HCFC-22",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
    "N2O",
    "NF3",
    "SF6",
    "SO2F2",
]

# #reactive_gases = ["SO2", "CO", "NH3", "VOC", "NOx"]
# #primary_aerosols = ["BC", "OC"]
# #secondary_aerosols = ["SO4", "NO3", "SOA"]

# reactive_gases = ["SO2", "CO", "VOC", "NOx", "NH3"]
# primary_aerosols = ["BC", "OC"]
# secondary_aerosols = ['SO4', 'NO3', 'SOA']

# emitted_species = wmghgs + reactive_gases + primary_aerosols
# species = wmghgs + reactive_gases + primary_aerosols + secondary_aerosols
# #species = wmghgs + reactive_gases + primary_aerosols

n_wmghgs = len(wmghgs)
# n_emitted_species = len(emitted_species)
# n_species = len(species)

n_configs = 1

In [ ]:
# grab some emissions
emissions_array = np.zeros((751, n_scenarios, n_configs, n_wmghgs, 1))
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for i_scenario, scenario in enumerate(scenarios):
    for i_wmghg, wmghg in enumerate(wmghgs):
        wmghg_rcmip_name = wmghg.replace("-", "")
#         if specie=='SO2':
#             specie_rcmip_name='Sulfur'
        emissions_array[:, i_scenario, 0, i_wmghg, 0] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+wmghg_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).squeeze().values.T
    
        # CO2 and N2O units need to behave
        if wmghg in ('CO2', 'N2O'):
            emissions_array[:, i_scenario, :, i_wmghg, :] = emissions_array[:, i_scenario, :, i_wmghg, :] / 1000
    
#     # fill non-directly emitted species with zeros. 
#     for ispec in range(n_emitted_species, n_species):
#         emissions_array[:, iscen, 0, ispec, 0] = 0

In [ ]:
emissions_array[5, 0, 0, :, 0]

In [ ]:
# RCMIP concentrations for benchmarking WMGHGs
concentration_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
for scenario in scenarios:
    concentration_rcmip[scenario] = {}
    for wmgas in wmghgs:
        gas_rcmip_name = wmgas.replace("-", "")
        concentration_rcmip[scenario][wmgas] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).values.squeeze()

In [ ]:
# grab indicative temperature projections for feedbacks
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp245-mean-temperature.csv')
ssp245_temperature_rfmip = df['temperature'].values

```
Emissions are          1750.5, 1751.5, 1752.5, ..., 2500.5  (751 array)
Concentrations are 1750.0, 1751.0, 1752.0, ...,   ..., 2501.0 (752 array)  
```

In [ ]:
concentration_array = np.ones((n_timebounds, n_scenarios, n_configs, n_wmghgs, 1)) * np.nan

In [ ]:
partition_fraction = {}
for wmghg in wmghgs:
    partition_fraction[wmghg] = np.zeros(N_GAS_BOXES)
    partition_fraction[wmghg][0] = 1
partition_fraction["CO2"] = np.array([0.2173, 0.2240, 0.2824, 0.2763])

lifetime = {
    "C2F6": 10000,
    "C3F8": 2600,
    "C4F10": 2600,
    "C5F12": 4100,
    "C6F14": 3100,
    "C7F16": 3000,
    "C8F18": 3000,
    "cC4F8": 3200,  # not standard PubChem name but used extensively in AR6
    "CCl4": 32,
    "CF4": 50000,
    "CFC-113": 93,
    "CFC-114": 189,
    "CFC-115": 540,
    "CFC-11": 52,
    "CFC-12": 102,
    "CH2Cl2": 0.493,
    "CH3Br": 0.8,
    "CH3CCl3": 5,
    "CH3Cl": 0.9,
    "CH4": 8.25,  # atmospheric burden lifetime in pre-industrial conditions. Source: Leach et al. (2021)
    "CHCl3": 0.501,
    "CO2": np.array([1e9, 394.4, 36.54, 4.304]),
    "Halon-1211": 16,
    "Halon-1301": 72,
    "Halon-2402": 28,
    "HCFC-141b": 9.4,
    "HCFC-142b": 18,
    "HCFC-22": 11.9,
    "HFC-125": 30,
    "HFC-134a": 14,
    "HFC-143a": 51,
    "HFC-152a": 1.6,
    "HFC-227ea": 36,
    "HFC-23": 228,
    "HFC-236fa": 213,
    "HFC-245fa": 7.9,
    "HFC-32": 5.4,
    "HFC-365mfc": 8.9,
    "HFC-4310mee": 17,
    "N2O": 123, #109,
    "NF3": 569,
    "SF6": 3200,
    "SO2F2": 36,
#     "SO2": 10/365.25,  # in gas phase
#     "CO":  1/6, # Khalil & Rasmussen 1990 (2 months); Zheng et al. 2019 (1-3 months)
#     "NH3": 1/6/365.25,  # Nair and Yu 2020: gas phase a few hours, particle form "under a week"; let's assume same as NOx
#     "NOx": 1/6/365.25,  # about 4 hours: https://acp.copernicus.org/articles/16/5283/2016/acp-16-5283-2016.pdf
#     "VOC": 1/6,  # days to a few months https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017gl072602; assume same as CO
#     "BC":  8/365.25,  # adjusted to get burden right: could be anything between 3 and 22 days according to Stjern et al. 2017
#     "OC":  8/365.25,  # Skeie et al 2020 suggests about 6 days
#     "SO4": 4.4/365.25,
#     "NO3": 4/365.25,  # Nair and Yu 2020: gas phase a few hours, particle form "under a week"; but burdens are to high
#     "SOA": 6.5/365.25,  # Hodzic et al. 2016
}

MOLWT = {
    "AIR": 28.97,  # reference?
    "C": 12.011,
    "C2F6": 138.01,
    "C3F8": 188.02,
    "C4F10": 238.03,
    "C5F12": 288.03,
    "C6F14": 338.04,
    "C7F16": 388.05,
    "C8F18": 438.06,
    "cC4F8": 200.03,  # not standard PubChem but used extensively in AR6
    "CCl4": 153.8,
    "CF4": 88.004,
    "CFC-113": 187.37,
    "CFC-114": 170.92,
    "CFC-115": 154.46,
    "CFC-11": 137.36,
    "CFC-12": 120.91,
    "CH2Cl2": 84.93,
    "CH3Br": 94.94,
    "CH3CCl3": 133.4,
    "CH3Cl": 50.49,
    "CH4": 16.043,
    "CHCl3": 119.37,
    "CO2": 44.009,
    "Halon-1211": 165.36,
    "Halon-1301": 148.91,
    "Halon-2402": 259.82,
    "HCFC-141b": 116.95,
    "HCFC-142b": 100.49,
    "HCFC-22": 86.47,
    "HFC-125": 120.02,
    "HFC-134a": 102.03,
    "HFC-143a": 84.04,
    "HFC-152a": 66.05,
    "HFC-227ea": 170.03,
    "HFC-23": 70.014,
    "HFC-236fa": 152.04,
    "HFC-245fa": 134.05,
    "HFC-32": 52.023,
    "HFC-365mfc": 148.07,
    "HFC-4310mee": 252.05,
    "N": 14.007,
    "N2": 28.014,
    "N2O": 44.013,
    "NF3": 71.002,
    "NO": 30.006,
    "NO2": 46.006,
    "NOx": 46.006,
    "S": 32.07,
    "SF6": 146.06,
    "SO2F2": 102.06,
#     "SO2": 64.069,
#     "SO4": 96.07,
#     "CO": 28.010,
#     "NH3": 17.031,
#     "NO3": 62.005,
#     "BC": 12.011,
#     "OC": 12.011,
#     "VOC": 59.7,  # weighted average, CEDS, 1970-2012
#     "SOA": 220, # guesstimate, Kourtchev et al 2015
}

iirf_0 = {wmghg: (lifetime[wmghg] * (1 - np.exp(-IIRF_HORIZON / lifetime[wmghg]))) for wmghg in wmghgs}
iirf_0['CO2'] = 29

iirf_uptake = {wmghg: 0 for wmghg in wmghgs}
iirf_uptake["CO2"] = 0.00846

iirf_temperature = {wmghg: 0 for wmghg in wmghgs}
#iirf_temperature = {gas: -iirf_0[gas]*0.015 for gas in gas_list}
iirf_temperature["CO2"] = 4.0
iirf_temperature["CH4"] = -0.3

iirf_airborne = {wmghg: 0 for wmghg in wmghgs}
iirf_airborne["CO2"] = 0.000819
iirf_airborne["CH4"] = 0.00032
iirf_airborne["N2O"] = -0.0065    # hand tuning to PD and PI lifetimes: Prather et al. 2015

concentration_per_emission = {}
for wmghg in wmghgs:
    concentration_per_emission[wmghg] = (
        1 / (M_ATMOS / 1e18 * MOLWT[wmghg] / MOLWT["AIR"])
    )

# for aerosol in primary_aerosols + secondary_aerosols:
#     concentration_per_emission[aerosol] = 1

baseline_emissions = {}
for wmghg in wmghgs:
    baseline_emissions[wmghg] = 0
baseline_emissions.update(
    {
        "CF4": 0.010071225,
        "CCl4": 0.024856862,
        "CH2Cl2": 246.6579,
        "CH3Br": 105.08773,
        "CH3Cl": 4275.7449,
        "CHCl3": 300.92479,
        "Halon-1211": 0.0077232726,
        # these are 1750 values
#         "SO2": 2.440048435,
#         "CO": 348.5273588,
#         "NH3": 6.927690091,
#         "NOx": 12.73521194,
#         "VOC": 60.02182622,
#         "BC": 2.097770755,
#         "OC": 15.44766815,
    }
)

In [ ]:
concentration_per_emission["Halon-1211"]  

In [ ]:
iirf_0["N2O"]

In [ ]:
wmghgs[21]

In [ ]:
lifetime_array                   = np.ones((1, 1, 1, n_wmghgs, N_GAS_BOXES)) * np.nan
partition_fraction_array         = np.ones((1, 1, 1, n_wmghgs, N_GAS_BOXES)) * np.nan
iirf_0_array                     = np.ones((1, 1, 1, n_wmghgs, 1)) * np.nan
iirf_airborne_array              = np.ones((1, 1, 1, n_wmghgs, 1)) * np.nan
iirf_temperature_array           = np.ones((1, 1, 1, n_wmghgs, 1)) * np.nan
iirf_uptake_array                = np.ones((1, 1, 1, n_wmghgs, 1)) * np.nan
concentration_per_emission_array = np.ones((1, 1, 1, n_wmghgs, 1)) * np.nan
baseline_emissions_array         = np.ones((1, 1, 1, n_wmghgs, 1)) * np.nan  # could be time dependent
baseline_concentration_array     = np.ones((1, n_scenarios, n_configs, n_wmghgs, 1)) * np.nan

# we would also loop/parallel over config here
for i_wmghg, wmghg in enumerate(wmghgs):
    lifetime_array                  [0, 0, 0, i_wmghg, :] = lifetime[wmghg]
    partition_fraction_array        [0, 0, 0, i_wmghg, :] = partition_fraction[wmghg]
    iirf_0_array                    [0, 0, 0, i_wmghg, :] = iirf_0[wmghg]
    iirf_airborne_array             [0, 0, 0, i_wmghg, :] = iirf_airborne[wmghg]
    iirf_temperature_array          [0, 0, 0, i_wmghg, :] = iirf_temperature[wmghg]
    iirf_uptake_array               [0, 0, 0, i_wmghg, :] = iirf_uptake[wmghg]
    concentration_per_emission_array[0, 0, 0, i_wmghg, 0] = concentration_per_emission[wmghg]
    baseline_emissions_array        [0, 0, 0, i_wmghg, 0] = baseline_emissions[wmghg]

# baseline concentration of WMGHGs
for i_scenario, scenario in enumerate(scenarios):
    for i_wmghg, wmghg in enumerate(wmghgs):
        baseline_concentration_array[0, i_scenario, 0, i_wmghg, 0] = concentration_rcmip[scenario][wmghg][0]

# CO2, CH4, N2O are not great in RCMIP
#baseline_concentration_array[0, :, 0, 21, 0] = 278.3
#baseline_concentration_array[0, :, 0, 19, 0] = 729.2
#baseline_concentration_array[0, :, 0, 39, 0] = 270.1
        
# baseline concentration of reactive gases and primary aerosols where we don't have RCMIP values are based on emissions & lifetime
## SO2
#baseline_concentration_array[0,:,:,43,0] = emissions_array[0,:,:,43,0] * lifetime_array[0,:,:,43,0] * concentration_per_emission["SO2"]

# for now, keep secondary baseline concentrations at zero
        
#    for ispec in range(n_wmghgs, n_emitted_species):
#        baseline_concentration_array[0, iscen, 0, ispec, 0] = 0
#        
# then all else is NaN for now
    
# # all SLCFs
# for ispec, specie in enumerate(reactive_gases+primary_aerosols):
#     baseline_concentration_array[0,:,:,ispec+n_wmghgs,0] = (
#         lifetime_array[0,:,:,ispec+n_wmghgs,0] * emissions_array[0,:,:,ispec+n_wmghgs,0] * concentration_per_emission[specie]
#     )

    
# for ispec, specie in enumerate(secondary_aerosols):
#     baseline_concentration_array[0,:,:,ispec+n_emitted_species,0] = 0
    
# fill in first timebound
concentration_array[0, ...] = baseline_concentration_array[0, ...]

In [ ]:
lifetime_array[0, 0, 0, 32, 0]

In [ ]:
# oxidation_matrix = np.zeros((53, 53))
# om_df = pd.DataFrame(oxidation_matrix, columns=species, index=species)
# #          from, to      oxidation efficiency   mass ratio of product to precursor   unit conversion
# om_df.loc["SO2", "SO4"] = 1.0                 * MOLWT["SO4"] / MOLWT["SO2"]
# om_df.loc["NH3", "NO3"] = 0.75                * MOLWT["NO3"] / MOLWT["NH3"]
# om_df.loc["VOC", "SOA"] = 0.75                * MOLWT["SOA"] / MOLWT["VOC"]
# om_df.loc["CH4", "CO2"] = 0                   * MOLWT["CO2"] / MOLWT["CH4"]          / 1000
# om_df.loc["VOC", "CO2"] = 0                   * MOLWT["CO2"] / MOLWT["VOC"]          / 1000
# om_df.loc["VOC", "CO"]  = 0                   * MOLWT["CO2"] / MOLWT["CO"]           / 1000
# om_df

In [ ]:
# oxidation_matrix = om_df.values
# om_df.loc["NH3", "NO3"], om_df.iloc[47, 51], oxidation_matrix[47, 51]

In [ ]:
# baseline concentration of CO is about 90 ppbv in Khalil & Rasmussen: here we have 11.6 ppb in PI
# but they were probably overestimating the total emissions

baseline_concentration_array[0,0,0,:,0]

In [ ]:
g1 = np.sum(
    partition_fraction_array * lifetime_array *
    (1 - (1 + IIRF_HORIZON/lifetime_array) *
    np.exp(-IIRF_HORIZON/lifetime_array)), axis=GAS_BOX_AXIS, keepdims=True
)

g0 = np.exp(
    -1 * np.sum(
        partition_fraction_array*lifetime_array*(1 - np.exp(-IIRF_HORIZON/lifetime_array)), axis=GAS_BOX_AXIS, keepdims=True
    )/g1
)

In [ ]:
g1[0,0,0,21,0], g0[0,0,0,21,0]  # looks right for CO2

In [ ]:
# we need separate baselines for both the forcing and concentrations at pre-industrial.

alpha_lifetime_array       = np.ones((n_timebounds, n_scenarios, 1, n_wmghgs, 1)) * np.nan
airborne_emissions_array   = np.ones((n_timebounds, n_scenarios, 1, n_wmghgs, 1)) * np.nan
gas_boxes_array            = np.zeros((1, n_scenarios, 1, n_wmghgs, N_GAS_BOXES))  # initial condition for restarts

cumulative_emissions_array = np.ones((n_timebounds, n_scenarios, 1, n_wmghgs, 1)) * np.nan
cumulative_emissions_array[0, ...] = 0  # initial condition: make interface option
cumulative_emissions_array[1:, ...] = np.cumsum(emissions_array * timestep, axis=TIME_AXIS)

airborne_emissions_array[0, ...] = 0  # initial condition: make interface option

In [ ]:
def calculate_alpha(
    airborne_emissions,
    cumulative_emissions,
    g0,
    g1,
    iirf_0,
    iirf_airborne,
    iirf_temperature,
    iirf_uptake,
    temperature,
    iirf_max=100,
):

    iirf = iirf_0 + iirf_uptake * (cumulative_emissions-airborne_emissions) + iirf_temperature * temperature + iirf_airborne * airborne_emissions
    iirf = (iirf>iirf_max) * iirf_max + iirf * (iirf<iirf_max)
    # overflow and invalid value errors occur with very large and small values
    # in the exponential. This happens with very long lifetime GHGs. Usually
    # these GHGs don't have a temperature dependence on IIRF but even if they
    # did the lifetimes are so long that it is unlikely to have an effect.
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        alpha = g0 * np.exp(iirf / g1)
        alpha[np.isnan(alpha)]=1
    return alpha

In [ ]:
def step_concentration(
    emissions,
    gas_boxes_old,
    airborne_emissions_old,
    alpha_lifetime,
    baseline_concentration,
    baseline_emissions,
    concentration_per_emission,
    lifetime,
#    oxidation_matrix,
    partition_fraction,
    timestep,
):

    decay_rate = timestep/(alpha_lifetime * lifetime)
    decay_factor = np.exp(-decay_rate)

#    print((1-decay_factor)[0,0,0,43,0], loss_this_timestep[0,0,0,43,0], gas_boxes_old[0,0,0,43,0])
    
#     # loss is in emissions units e.g. Tg
#     loss_this_timestep = (gas_boxes_old * (1-decay_factor))
#     #print(gas_boxes_old.shape)
#     emissions_from_oxidation = np.sum(
#         loss_this_timestep[:, :, :, :, None, :] * oxidation_matrix[None, None, None, :, :, None],
#         axis=(3, 5)
#     )    
    
    #print(loss_this_timestep[0, 0, 0, 28, :], emissions[0, 0, 28], decay_factor[0, 0, 0, 28, 0])
    # additions and removals
    gas_boxes_new = (
        partition_fraction *
        (emissions - baseline_emissions) *
        1 / decay_rate *
        (1 - decay_factor) * timestep + gas_boxes_old * decay_factor
    )
#     print(loss_this_timestep[0,0,0,43,0], gas_boxes_old[0,0,0,43,0], gas_boxes_new[0,0,0,43,0], 1-decay_factor[0,0,0,43,0])
        
    airborne_emissions_new = np.sum(gas_boxes_new, axis=GAS_BOX_AXIS, keepdims=True)
    concentration_out = baseline_concentration + concentration_per_emission * airborne_emissions_new

    return concentration_out, gas_boxes_new, airborne_emissions_new

In [ ]:
concentration_per_emission_array[0,0,0,42,0]

In [ ]:
gas_boxes_array = np.zeros((1, n_scenarios, 1, n_wmghgs, N_GAS_BOXES))  # initial condition for restarts

for i_timestep in range(n_timesteps):  # 0 to 751
    alpha_lifetime_array[i_timestep, ...] = calculate_alpha(   # this timestep
        airborne_emissions_array[i_timestep, ...],  # last timebound
        cumulative_emissions_array[i_timestep, ...],  # last timebound
        g0,
        g1,
        iirf_0_array,
        iirf_airborne_array,
        iirf_temperature_array,
        iirf_uptake_array,
        ssp245_temperature_rfmip[i_timestep],  # last timebound
        IIRF_MAX
    )
    concentration_array[i_timestep+1, ...], gas_boxes_array, airborne_emissions_array[i_timestep+1, ...] = step_concentration( # next timebound
        emissions_array[i_timestep, ...],  # this timestep
        gas_boxes_array, # last timebound
        airborne_emissions_array[i_timestep, ...],  # last timebound
        alpha_lifetime_array[i_timestep, ...],
        baseline_concentration_array,
        baseline_emissions_array,
        concentration_per_emission_array,
        lifetime_array,
#        oxidation_matrix,
        partition_fraction_array,
        timestep,
    )

In [ ]:
pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 0, 21, 0])
pl.plot(np.arange(1750.5, 2022), concentration_rcmip['ssp245']['CO2'][:272])
concentration_array[270, 0, 0, 21, 0]

In [ ]:
#concentration_array[-100:, 0, 0, 28, 0]

In [ ]:
baseline_concentration_array[0, 0, 0, :, 0]

In [ ]:
pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 0, 39, 0])
pl.plot(np.arange(1750.5, 2022), concentration_rcmip['ssp245']['N2O'][:272])

In [ ]:
# pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 0, 43, 0])
# pl.title('SO2 concentration')
# pl.ylabel('ppb')
# concentration_array[250, 0, 0, 43, 0]   # this is close to Gunnar's value of 0.9

In [ ]:
# pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 0, 44, 0])
# pl.title('CO concentration')
# pl.ylabel('ppb')
# concentration_array[240, 0, 0, 44, 0]   # Khalil & Rasmussen say 90 Tg in 1990 - this is OK

In [ ]:
# pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 0, 45, 0])
# pl.title('VOC concentration')
# pl.ylabel('ppb')
# concentration_array[250, 0, 0, 45, 0]  # this isn't far off the 16 Tg here: https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017gl072602

In [ ]:
# pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 0, 46, 0])
# pl.title('NOx concentration')
# pl.ylabel('ppb')
# concentration_array[250, 0, 0, 46, 0]  # this isn't far off the 16 Tg here: https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017gl072602

In [ ]:
# pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 0, 47, 0])
# pl.title('BC burden')
# pl.ylabel('Tg')
# concentration_array[260, 0, 0, 47, 0] - concentration_array[0, 0, 0, 47, 0]  # Skeie et al. 2011: 153 GgC in 2010 versus 1750. Perfect.

In [ ]:
# pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 0, 48, 0])
# pl.title('OC burden')
# pl.ylabel('Tg')
# concentration_array[260, 0, 0, 48, 0] - concentration_array[0, 0, 0, 48, 0]  # Skeie et al. 2011: 562 Gg in 2010 versus 1750, but off higher 1750 emissions

In [ ]:
# pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 0, 49, 0])
# pl.title('NH3 concentration')
# pl.ylabel('ppb')
# concentration_array[260, 0, 0, 49, 0] - concentration_array[0, 0, 0, 49, 0]  # Skeie et al. 2011: 562 Gg in 2010 versus 1750, but off higher 1750 emissions

In [ ]:
# pl.plot(np.arange(1750, 2502), concentration_array[:, 0, 0, 50, 0])
# pl.title('SO4 burden')
# pl.ylabel('Tg')
# concentration_array[260, 0, 0, 50, 0] # Skeie et al. 2011: 562 Gg in 2010 versus 1750, but off higher 1750 emissions
# concentration_array[-100:, 0, 0, 50, 0]

In [ ]:
# pl.plot(np.arange(1750, 2022), concentration_array[0:272, 0, 0, 50, 0])
# pl.title('SO4 burden')
# pl.ylabel('Tg')
# concentration_array[260, 0, 0, 50, 0] # Skeie et al. 2011: 562 Gg in 2010 versus 1750, but off higher 1750 emissions

In [ ]:
# # steady state concentration

# lamb = 4/365.25
# emis = 2.440048435 * MOLWT["SO4"]/MOLWT["SO2"] # conversion, then per day

# lamb * emis
# #np.exp(-1/lamb)

In [ ]:
# concentration_array[240, 0, 0, 43:, 0]

In [ ]:
# pl.scatter(emissions_array[:,0,0,43,0], concentration_array[1:,0,0,43,0])

In [ ]:
# pl.scatter(emissions_array[:,0,0,44,0], concentration_array[1:,0,0,44,0])

In [ ]:
for i_scenario, scenario in enumerate(scenarios):
    pl.plot(timebounds, concentration_array[:,i_scenario,0,21,0], label=scenario)
pl.legend();

In [ ]:
pl.plot(np.arange(1750, 2502), cumulative_emissions_array[:, 0, 0, 21, 0])

In [ ]:
concentration = {}

for i_scenario, scenario in enumerate(scenarios):
    concentration[scenario] = {}
    for i_wmghg, wmghg in enumerate(wmghgs):
        concentration[scenario][wmghg] = concentration_array[:, i_scenario, 0, i_wmghg, 0]

In [ ]:
for iscen, scenario in enumerate(scenarios):
    fig, ax = pl.subplots(6, 8, figsize=(16,16))
    for i_wmghg, wmghg in enumerate(wmghgs):
        iy = i_wmghg % 8
        ix = i_wmghg // 8
        ax[ix, iy].plot(timebounds, concentration[scenario][wmghg], label='FaIR 2.1', color='k')
        ax[ix, iy].plot(timebounds[:-1], concentration_rcmip[scenario][wmghg], label='History + MAGICC6', color='r')
        ax[ix, iy].set_title(wmghg)
    fig.tight_layout()

In [ ]:
alpha_lifetime_array[:, 0, 0, 39, 0] * lifetime_array[0, 0, 0, 39, 0]

In [ ]:
alpha_lifetime_array[258, 0, 0, 39, 0] * lifetime_array[0, 0, 0, 39, 0]  # target 109

In [ ]:
alpha_lifetime_array[100, 0, 0, 39, 0] * lifetime_array[0, 0, 0, 39, 0]  # target 123

In [ ]:
mkdir_p("../data/output/")
with open("../data/output/rcmip-fair21-concentrations.pkl","wb") as fileout:
    pickle.dump(concentration, fileout)